# 라이브러리

In [ ]:
! pip install konlpy 

In [67]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import warnings
warnings.filterwarnings(action='ignore')

# matplotlib 그래프 속성 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['figure.figsize'] = (12,16)
plt.rcParams['font.size'] = 20

# 그래프 그릴 때 마이너스(-) 부분도 표시해주기 (로그오즈비 표현을 위해 세팅)
plt.rcParams['axes.unicode_minus'] = False

# 사용자 정의 함수

In [51]:
# 텍스트 전처리
def text_preprocess(text):

  # 숫자, 기호, 영문 제거
  new_text = re.sub(r'[^가-힣]', ' ', text)

  # 불필요한 품사 제거
  okt = Okt()
  prepro_text = []
  for word, tag in okt.pos(new_text):
    if tag not in ['Josa', 'Suffix']:
      prepro_text.append(word)
  
  result = [word for word in prepro_text if word not in stop_words]

  #result = ' '.join(prepro_text)

  return result


  # Counter 빈도분석
def analysis_by_counter(doc, display_num):
  prepro_words = []
  for text in doc:
    prepro_words += text
    
  count = Counter(prepro_words)

  count_word = [word for word, vec in count.most_common(display_num)]
  count_vec = [vec for word, vec in count.most_common(display_num)]

  plt.barh(count_word[::-1], count_vec[::-1], color='tab:pink')
  plt.yticks(count_word[::-1])
  plt.show()

  return count_word, count_vec, count


# CountVectorizer 빈도 분석
def analysis_by_CountVectorizer(texts, count_vec, display_num, color, title):
    
    texts = texts.map(lambda x: ' '.join(x))

    count_vec.fit(texts)
    
    word_dict = sorted(count_vec.vocabulary_.items())
    idx2word = {idx:word for word, idx in word_dict}

    total_text = []
    total_text.append(' '.join(texts.values))

    count_matrix = count_vec.transform(total_text)

    count_word = []
    count_vector = []

    for i in range(display_num,0,-1):
        count_word.append(idx2word[(-count_matrix.toarray()[0]).argsort()[i-1]])
        count_vector.append(count_matrix.toarray()[0][(-count_matrix.toarray()[0]).argsort()[i-1]])

    print(count_word)
    print(count_vector)

    title = title + ' 빈도 분석'

    plt.barh(count_word, count_vector, color=color)
    plt.yticks(count_word)
    plt.title(title)
    plt.show()

    return count_word, count_vector

# 데이터 로드 & 생성

In [40]:
# 파일 경로
old_path = '/content/drive/MyDrive/Colab Notebooks/블로그포스팅/쉬운성경/구약'
new_path = '/content/drive/MyDrive/Colab Notebooks/블로그포스팅/쉬운성경/신약'

old_testament = os.listdir(old_path)
new_testament = os.listdir(new_path)

# 순서대로 정렬이 안 돼 있어서 정렬
new_testament.sort()

In [41]:
# 구약
old_testament_df = pd.DataFrame(columns=['성경제목', '성경주소', '성경본문'])
for title in old_testament:
    each_old = old_path + '/' + title
    df = pd.read_table(each_old, encoding='cp949', names=['성경본문'])
    df['성경제목'] = title.split()[1].strip('.txt')
    df['성경본문'] = df['성경본문'].astype(str)
    df['성경주소'] = df['성경본문'].map(lambda x : x[ : x.find(' ')])
    df['성경본문'] = df['성경본문'].map(lambda x : x[x.find(' ')+1 :])
    df = df[['성경제목', '성경주소', '성경본문']]
    old_testament_df = pd.concat([old_testament_df, df], axis=0, ignore_index=True)

# 신약
new_testament_df = pd.DataFrame(columns=['성경제목', '성경주소', '성경본문'])
for title in new_testament:
    each_new = new_path + '/' + title
    df = pd.read_table(each_new, encoding='cp949', names=['성경본문'])
    df['성경제목'] = title.split()[1].strip('.txt')
    df['성경본문'] = df['성경본문'].astype(str)
    df['성경주소'] = df['성경본문'].map(lambda x : x[ : x.find(' ')])
    df['성경본문'] = df['성경본문'].map(lambda x : x[x.find(' ')+1 :])
    df = df[['성경제목', '성경주소', '성경본문']]
    new_testament_df = pd.concat([new_testament_df, df], axis=0, ignore_index=True)

# 성격 각 권별 텍스트 분석

In [42]:
# 구약 성경 리스트
old_testament_list = []
for title in old_testament:
  old_testament_list.append(title[title.find(' ') + 1:title.find('.')])

# 신양 성경 리스트
new_testament_list = []
for title in new_testament:
  new_testament_list.append(title.split()[1].strip('.txt'))

## 빈도 분석

In [ ]:
# 1번째 방법 : Counter 사용

# 창세기로 sample example
for title in old_testament_list[0:1]:
  
  # 각 성경별 본문 시리즈로 불러오기
  each_old_testament = old_testament_df['성경본문'][old_testament_df['성경제목']== title]

  # 불용어 사전
  stop_words = ['하셨습니다', '그런데', '그', '그러자', '있는', '아니었고', '없이', '했습니다', '것', '내', '이',
                '입니다', '너', '네', '저', '나', '해', '그리고']

  # 본문별 전처리
  each_old_testament = each_old_testament.map(text_preprocess)

  # 빈도분석 실행
  counter_word, countwr_vec, counter = analysis_by_counter(each_old_testament, 20)

  # counter는 리스트 안에 튜플 형식으로
  # [(말, 450), (사람, 385), ...]

In [ ]:
# 2번째 방법 : CountVectorizer 사용

# 창세기로 sample example
for title in old_testament_list[0:1]:
  
  # 각 성경별 본문 시리즈로 불러오기
  each_old_testament = old_testament_df['성경본문'][old_testament_df['성경제목']== title]

  # 본문별 전처리
  each_old_testament = each_old_testament.map(text_preprocess)

  # 불용어 사전
  stop_words = ['하셨습니다', '그런데', '그', '그러자', '있는', '아니었고', '없이', '했습니다', '것', '내', '이',
                '입니다', '너', '네', '저', '나', '해', '그리고']

  # CountVectorizer 이용 빈도분석
  # vectorizer
  cnt_vectorizer = CountVectorizer(
      stop_words = stop_words,
      ngram_range = (1,1),
      max_features = 100,
      min_df = 50,
      #max_df =30
  )

  # analysis
  cntvec_word, cntvec_vec = analysis_by_CountVectorizer(each_old_testament,cnt_vectorizer, 20, 'tab:blue', title)

  # 워드클라우드에 맞게 데이터 모양 바꾸기
  [(word,vec) for word, vec in zip(cntvec_word, cntvec_vec)]

# 워드클라우드

## 기본

In [ ]:
# Counter로 워드클라우드

# 워드 클라우드 객체 생성
wordcloud = WordCloud(
    font_path = 'NanumGothic',
    width = 400,
    height = 400,
    background_color = 'white',
    max_words = 50
)

# 데이터 적합
wordcloud = wordcloud.generate_from_frequencies(counter)

# 이미지 출력
plt.figure(figsize=(8,8))
plt.imshow(wordcloud.to_image())
plt.axis('off')
plt.show()

In [ ]:
# CountVectorizer로 워드클라우드

# WordCloud 객체
cnt_wordcloud = WordCloud(
    font_path = 'NanumGothic',
    width = 400,
    height = 400,
    background_color = 'white',
)

wordvec_package = { word:vec for word, vec in zip(cntvec_word, cntvec_vec)}

# 데이터 적합
cnt_wordcloud = cnt_wordcloud.generate_from_frequencies(wordvec_package)

# 이미지 출력
plt.figure(figsize=(8,8))
plt.imshow(cnt_wordcloud.to_image())
plt.axis('off')
plt.show()